<a href="https://colab.research.google.com/github/MahreenAthar/MLOps-Assignment-2/blob/BranchMahreen/Model_StockExchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import joblib
from flask import Flask, request, jsonify, render_template
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import yfinance as yf

In [3]:
app = Flask(__name__, template_folder='/app')

# Loading the dataset

In [9]:
ticker = yf.Ticker("GOOG")
data = ticker.history(start=datetime.date(2020, 1, 1), end=datetime.date.today())
data.reset_index(inplace=True)

# Splitting Data
80% training, 20% testing

In [10]:
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Define input features 
Open, High, Low, and Volume

In [11]:
X_train = train_data[['Open', 'High', 'Low', 'Volume']]
X_test = test_data[['Open', 'High', 'Low', 'Volume']]

# Define target variable (Close)

In [12]:
y_train = train_data['Close']
y_test = test_data['Close']

# Create polynomial features of degree 2

In [13]:
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train polynomial regression model

In [14]:
model = LinearRegression()
model.fit(X_train_poly, y_train)

LinearRegression()

# Print mean squared error on test data


In [15]:
y_pred = model.predict(X_test_poly)
mse = mean_squared_error(y_test, y_pred)

# Save the trained model

In [16]:
joblib.dump(model, 'model.pkl')

['model.pkl']

# Load the trained model

In [17]:
model = joblib.load('model.pkl')